In [2]:
%pip install Pyarrow
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Generate x_train, x_test, y_train, y_test

In [3]:
from PIL import Image
import numpy as np
import glob

def get_list_of_files(source_path:str) -> list[str]:
	train_files= glob.glob(source_path+'train*.png')
	train_files = [(int(i[len(source_path)+5:-4]), i) for i in glob.glob(source_path+'train*.png')]
	train_files.sort()
	return [i[1] for i in train_files]

In [8]:
pngs = get_list_of_files('../../input/images/2000-samples/')
x = np.array([np.asarray(Image.open(i))[:,:,:3] for i in pngs])
print(x.shape)

x = x.reshape(x.shape[0], -1)
x.shape

(2000, 255, 255, 3)


(2000, 195075)

In [9]:
import pandas

def y_int_to_str(y:int) -> str:
	match y:
		case 0: return "No Whale"
		case 1: return "Whale Present"
		case _: raise ValueError("Unrecognized label")
print(y_int_to_str(1))

y = pandas.read_csv('../../input/audio/data/train.csv')
y = y["label"][:len(x)]
y.shape

Whale Present


(2000,)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
	x,
	y,
	test_size=0.33,
	random_state=4,
	stratify=y
)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1340, 195075), (660, 195075), (1340,), (660,))

# K-NN

## Basic K-NN

## Cross-validation to find the best k